# EDA


## Loading big dataset

### Imports

In [193]:
import pandas as pd
import seaborn as sns
import numpy as np
import string
import tensorflow as tf
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer

### Loading data

In [194]:
data=pd.read_csv('../raw_data/training_data.csv',  header=None, encoding='latin-1')
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [195]:
#set names to columns
data.columns=['label','id', 'date','query','username','tweet']


In [196]:
# drop columns
data=data.drop(columns=['id', 'date','query','username'])


In [197]:
#check the balance of the classes
data.label.value_counts()

0    800000
4    800000
Name: label, dtype: int64

## Little split


In [261]:
small=data.sample(n=500)
small.label=small.label*0.25
small.label=small.label.astype(int)


# Preprocessing

## Basic cleaning

In [215]:
#In this function we lower case everything, remove numbers puntuation and stopwords and strip the text
def basic_cleaning(sentence):
    sentence = sentence.lower()
    sentence = ''.join(char for char in sentence if not char.isdigit())
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') 
    sentence = sentence.strip()
    word_tokens = word_tokenize(sentence)
    sentence = [w for w in word_tokens if not w in stop_words]
    sentence= [WordNetLemmatizer().lemmatize(word, pos = "v")  # v --> verbs
              for word in sentence]
    snetence=[WordNetLemmatizer().lemmatize(word, pos = "n")  # v --> verbs
              for word in sentence]
    return ' '.join(word for word in sentence)

def chunk_cleaning(chunk):
    stop_words = stopwords.words('english')
    stop_words.append('u')
    stop_words.append('r')
    stop_words=set(stop_words)
    small_cleaned=[basic_cleaning(tweet) for tweet in chunk]
    return small_cleaned
     


In [262]:
train=pd.DataFrame(data=np.array([chunk_cleaning(small.tweet[:300]),small.label[:300]]).T,columns=['tweet','label'])
test=pd.DataFrame(data=np.array([chunk_cleaning(small.tweet[300:400]),small.label[300:400]]).T,columns=['tweet','label'])

In [217]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

In [218]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Model

In [209]:

def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
    train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

    validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
    return train_InputExamples, validation_InputExamples

train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                               test, 
                                                                               'tweet', 
                                                                               'label')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'tweet'
LABEL_COLUMN = 'label'

In [263]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/Users/alvarodelamaza/.pyenv/versions/3.8.12/envs/crowdfeel/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [264]:
test

,tweet,label
0,room wall bore want paint put moooore posters ...,0
1,beckyj yourmomb thank guy,1
2,igii tweet ur alive,1
3,random fact audition video lenses glass xd htt...,1
4,work day meh money towards macbook,0
...,...,...
95,soak sunshine,1
96,lucindaaxo aha okay well give ring somthing je...,1
97,keep gettin wave nausea fun badim scar eat ill...,0
98,recruitlocal thx ff love awesome weekend,1


In [265]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
20/20 [==============================] - 264s 12s/step - loss: 0.7441 - accuracy: 0.4700 - val_loss: 0.6937 - val_accuracy: 0.4900
Epoch 2/2
20/20 [==============================] - 232s 12s/step - loss: 0.7032 - accuracy: 0.5167 - val_loss: 0.6929 - val_accuracy: 0.5100


In [295]:
pred_sentences=chunk_cleaning(small.tweet[400:401].values.tolist())
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model.predict(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
#for i in range(len(pred_sentences)):
    #print(pred_sentences[i], ": \n", labels[label[i]], np.array(small.label[400:500])[i])


ValueError: Cannot generate a hashable key for IteratorSpec(({'input_ids': TensorSpec(shape=(None, 21), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, 21), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 21), dtype=tf.int32, name=None)},),) because the _serialize() method returned an unsupproted value of type <class 'transformers.tokenization_utils_base.BatchEncoding'>

In [290]:

np.array(small.tweet)

array(['@cathyjh the moss wall  (i want one) and the black board. hehehe i would like to say i drew the shark but i didnt. *evil laugh*',
       '@dancebabydance Happy Birthday marissa!!! hope you have a incredible b-day ',
       'I need a drink!!! Fck, fck, fck! ',
       '@niffany15  What should I feed my dino?',
       "@DonnieWahlberg: What happened to the Asian love on the unofficial video? I guess mine didn't make the cut. ",
       'burger.jpg: Shared by Bayou Whaaaaaa, peut etre plus gras que le welsh  http://tinyurl.com/dg7um5',
       '@FUCKCITY we all do  good luck getting tatooed next week',
       '@adwordpro follow _MoisesArias !!! He is the real moises arias!  tnx',
       'zofran is not making this work day any easier, or my stomach any less upset ',
       '@happylovesChuck I have stopped twiving!  my heart hurts',
       '@chelsea_playboy R.I.P curls ',
       '@theBrandiCyrus OMG! me tooo. letss goo togethherrr ',
       'Against my advice @strikerobi put scented lo

In [274]:
tf_batch

{'input_ids': <tf.Tensor: shape=(100, 28), dtype=int32, numpy=
array([[  101, 11576,  2378, ...,     0,     0,     0],
       [  101,  5772, 16558, ...,     0,     0,     0],
       [  101,  6805,  6402, ...,     0,     0,     0],
       ...,
       [  101, 18168,  2290, ...,     0,     0,     0],
       [  101,  2131,  2079, ...,     0,     0,     0],
       [  101, 16593,  2131, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(100, 28), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(100, 28), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}